# Keras Tuner Bars

In [1]:
import h5py
import numpy as np
import tensorflow
from tensorflow import keras
import kerastuner

print("tensorflow", tensorflow.__version__)
print("keras", keras.__version__)
print("keras-tuner", kerastuner.__version__)

tensorflow 2.1.0
keras 2.2.4-tf
keras-tuner 1.0.1


In [2]:
class HDF5DataGeneratorBars(keras.utils.Sequence):
    def __init__(self, file, num_classes, batch_size=200, mode='train', train_test_split=0.8):
        self.file = h5py.File(file, "r")
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.mode = mode

        batches = int(np.floor(self.file["multiplicity"].len() / self.batch_size)
        if self.mode == 'train':
            self.num_batches = int(batches * train_test_split)
            self.offset = 0
        elif self.mode == 'test':
            self.num_batches = int(batches * (1 - train_test_split))
            self.offset = int(batches * train_test_split)
        else:
            raise

    def __len__(self):
        return self.num_batches

    def __getitem__(self, index):
        x, y = self.getbatch(index)
        np.place(y, y >= self.num_classes, self.num_classes - 1)  # FIXME
        y = keras.utils.to_categorical(y, num_classes=self.num_classes)
        return x, y

    def getbatch(self, index):
        "Generate one batch of data"
        start = self.batch_size * (index + self.offset)
        end = self.batch_size * (index + self.offset + 1)
        x1 = np.array(self.file["flatfeatures"][start:end])
        x2 = np.array(self.file["consolidated"][start:end])
        x = np.append(x1, x2, axis=1)
        # multiplicity column 2 -> label = 'nPH'
        y = np.array(self.file["multiplicity"][start:end, [2]].ravel())
        return x, y

In [3]:
class Model(kerastuner.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.models.Sequential()
        model.add(
            keras.layers.Dense(
                units=hp.Int(f"units_1", min_value=1000, max_value=10000, step=1000), activation="relu",
            )
        )
        model.add(
            keras.layers.Dense(
                units=hp.Int(f"units_2", min_value=600, max_value=2000, step=200), activation="relu",
            )
        )
#         model.add(
#             keras.layers.Dense(
#                 units=hp.Int(f"units_3", min_value=50, max_value=500, step=50), activation="relu",
#             )
#         )
        model.add(keras.layers.Dense(units=self.num_classes, activation="softmax"))

        optimizer = keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3]))
        loss = keras.losses.CategoricalCrossentropy()

        model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

        return model

In [4]:
num_dp = 30
max_neutrons = 5
num_classes = max_neutrons + 1

file = f"data/600AMeV_{num_dp}dp.bars-shuffled.h5"
generator = HDF5DataGeneratorBars(file, num_classes=num_classes, mode='train')
test_generator = HDF5DataGeneratorBars(file, num_classes=num_classes, mode='test')
model = Model(max_neutrons)

tuner = kerastuner.tuners.RandomSearch(
    model,
    objective="accuracy",
    max_trials=5,
    executions_per_trial=3,
    directory="tmp-kerastuner-V2",
    project_name="tunebars1",
)
tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project tmp-kerastuner-V2\tunebars1\oracle.json
INFO:tensorflow:Reloading Tuner from tmp-kerastuner-V2\tunebars1\tuner0.json


In [5]:
tuner.search(
    generator,
    #epochs=20,
    #validation_data=(x_val, y_val),
    verbose=2,
)

INFO:tensorflow:Oracle triggered exit


In [6]:
tuner.results_summary()